Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

In [1]:
%%capture
import sys

if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/bsmrvl/DS-Unit-2-Applied-Modeling/tree/master/data/'
    !pip install category_encoders==2.*

else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np
import matplotlib.pyplot as plt

from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import plot_roc_curve, classification_report

In [3]:
df = pd.read_csv(DATA_PATH + 'ABIII_English.csv', dtype=object, parse_dates=['date'])

In [4]:
def nulls(df):
    nulls = df.isnull().sum()
    return nulls[nulls>0]

In [5]:
## Create combined target. Remove the ~5% of data that doesn't fall into the top
## three classes.

# df['target'] = (df['q301'] + df['q302']).replace({'YesYes':'Voted, attended campaign activities',
#                                                   'YesNo':'Voted, did not attend campaign activities',
#                                                   'NoNo':'Neither voted nor attended activities'})
# target_options = ['Voted, attended campaign activities',
#                   'Voted, did not attend campaign activities',
#                   'Neither voted nor attended activities']

# df = df[df['target'].isin(target_options)]

# df = df.drop(columns=['q301', 'q302'])
# df = df.drop(columns=['qid','bid'])
# df = df.drop(columns=['q1002','wt','form','samp'])

In [6]:
df['target'] = df['q301']
target_options = ['Yes','No']
df = df[df['target'].isin(target_options)]
df['target'].value_counts(normalize=True)

Yes    0.582516
No     0.417484
Name: target, dtype: float64

In [7]:
df_VI = df.loc[:,'q6012':'q618']

In [8]:
df_VI['q608'] = df_VI['q608a'].replace(np.NaN, '') + df_VI['q608b'].replace(np.NaN, '')
df_VI = df_VI.drop(columns=['q608a','q608b'])

In [9]:
df_VI['q617'] = df_VI['q617'].replace(np.NaN, 'Missing')
df_VI['q618'] = df_VI['q618'].replace(np.NaN, 'Missing')

In [10]:
df_VII = df.loc[:,'q7001':'q713']
nulls(df_VII)

q7006       2418
q7007       6469
q7008       6551
q7009      10066
q70010     10066
q70011     13674
q70012     13447
q70013     13447
q70014     13447
q70015     13447
q70016     13447
q700a6      2418
q700a7      5277
q700a8      5359
q700a9     10066
q700a10    10066
q700a11    13674
q700a12    13447
q700a13    13447
q700a14    13447
q700a15    13447
q700a16    13447
q701c       3465
q701d1      2255
q701d2      2255
q708a      10516
q708kw     13674
dtype: int64

In [11]:
df_VII = df_VII.dropna(thresh=10000, axis=1)
nulls(df_VII)

q7006     2418
q700a6    2418
q701c     3465
q701d1    2255
q701d2    2255
dtype: int64

In [12]:
df_VII = df_VII.replace(np.NaN, 'Not asked')

In [13]:
df_V = df.loc[:,'q5012':'q530']
nulls(df_V)

q5016      2802
q5017      5717
q50111    12308
q5152         7
q518a2     7345
q518b2     7343
q5202       791
q5203       791
q5204       791
q5206       791
q5207       791
q5208       791
q5224      1192
dtype: int64

In [14]:
df_V['q5182'] = df_V['q518a2'].replace(np.NaN, '') + df_V['q518b2'].replace(np.NaN, '')
df_V = df_V.drop(columns=['q518a2','q518b2'])
nulls(df_V)

q5016      2802
q5017      5717
q50111    12308
q5152         7
q5202       791
q5203       791
q5204       791
q5206       791
q5207       791
q5208       791
q5224      1192
dtype: int64

In [15]:
df_V = df_V.dropna(thresh=10000, axis=1)
nulls(df_V)

q5016    2802
q5152       7
q5202     791
q5203     791
q5204     791
q5206     791
q5207     791
q5208     791
q5224    1192
dtype: int64

In [16]:
df_V = df_V.replace(np.NaN, 'Not asked')

In [17]:
df_II = df.loc[:,'q2011':'q240']
nulls(df_II)

q20114       12486
q20115       13498
q20116       13498
q20117       13674
q20118       13674
q20119       13447
q202_insh     3615
q2035         9895
q2037        13447
q20416       13498
q20417       13498
q20418       13498
q20419       13674
q2062           51
q210             1
q210a         2661
q211          2661
q211_insh     5448
dtype: int64

In [18]:
df_II = df_II.dropna(thresh=8000, axis=1)
nulls(df_II)

q202_insh    3615
q2062          51
q210            1
q210a        2661
q211         2661
q211_insh    5448
dtype: int64

In [19]:
df_II = df_II.replace(np.NaN, 'Not asked')

In [20]:
df_IV = df.loc[:,'q402':'q4115']
nulls(df_IV)

q4063    1014
q4101    7277
q4102    7277
q4103    7277
q4111    7278
q4112    7277
q4113    7277
q4114    7277
q4115    7277
dtype: int64

In [21]:
df_IV = df_IV.replace(np.NaN, 'Not asked')
df_IV = df_IV.drop(columns='q404')

In [22]:
X = pd.concat([df_II, df_IV, df_V, df_VI, df_VII], axis=1)
y = df['target']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [24]:
model = make_pipeline(
    OrdinalEncoder(),
    RandomForestClassifier(n_estimators=500, max_leaf_nodes=None, random_state=42)
)

model.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['q2011', 'q2013', 'q2014', 'q2016',
                                      'q2017', 'q20112', 'q20113', 'q202',
                                      'q202_insh', 'q2031', 'q2032', 'q2033',
                                      'q2034', 'q2042', 'q2043', 'q2044',
                                      'q20412', 'q2054', 'q2055', 'q2061',
                                      'q2062', 'q210', 'q210a', 'q211',
                                      'q211_insh', 'q213', 'q214', 'q216',
                                      'q217', 'q2185', ...],
                                mapping=[{'col': 'q2011',
                                          'data_type': dtype('O')...
Refuse                 6
NaN                   -2
dtype: int64},
                                         {'col': 'q217',
                                          'data_type': dtype('O'),
                                          'mapping': No          

In [25]:
model.score(X_test, y_test)

0.6848196051735874

In [26]:
# fig, ax = plt.subplots(figsize=[6,6])

# plot_roc_curve(model, X_test, y_test, ax=ax)

# plt.show()

In [27]:
labels = X.columns
importances = model.named_steps['randomforestclassifier'].feature_importances_
important_df = pd.Series(importances, index=labels)

In [28]:
important_df.sort_values()

q50110    0.001092
q5014     0.001892
q5015     0.001917
q5012     0.001995
q501b     0.002188
            ...   
q512      0.010683
q240      0.010868
q530      0.011311
q503      0.012211
q4061     0.014406
Length: 169, dtype: float64

In [29]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

          No       0.69      0.44      0.54      1225
         Yes       0.68      0.86      0.76      1713

    accuracy                           0.68      2938
   macro avg       0.69      0.65      0.65      2938
weighted avg       0.69      0.68      0.67      2938

